[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevin7261/Geographic-Data-Science-with-Python/blob/main/登革熱資料轉換.ipynb)

# 登革熱資料轉換

https://data.cdc.gov.tw/zh_TW/group/dengue

https://data.cdc.gov.tw/zh_TW/dataset/dengue-daily-determined-cases-1998/resource/e868ae05-2381-44f2-9656-42292ef7e0c6


## [1] 讀取原始資料

In [ ]:
# @title 初始值設定

PROJECT_NAME = "15_台南市區_合併位置"
TOWNSHIPS_LIST = ["中西區", "東區", "南區", "北區", "安平區", "安南區"]
GSHEET_NAME = "Dengue Daily"
DENGUE_DAILY_CSV_PATH = "https://drive.google.com/uc?export=download&id=1hWD2AiA-_oXPRZt5FmuRGsRrlD7rgT3N"
SHP_FILE_PATH = "https://drive.google.com/file/d/1hB8_r7i-7uqxVkqy7DzN8Kl5px6XLsVn/view?usp=sharing"
GDRIVE_PATH = "https://drive.google.com/drive/folders/1g39YQg2LLE5nf1YVDO4SUhgwa8VUaETF?usp=sharing"

In [ ]:
# @title 掛載Google雲端硬碟

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 安裝套件

# ✅ 最安全方式：一次安裝所有套件 + 修正 google-auth 相容性
!pip install -q gdown gspread gspread_dataframe pyproj
!pip install -q google-auth==2.38.0  # 降版以相容 colab
!pip install -q PyDrive

## [2] 建立Google Sheet


In [ ]:
# @title create_gsheet

import gspread
from google.colab import auth
import pandas as pd
from gspread_dataframe import set_with_dataframe
from google.auth import default
from google.auth.transport.requests import Request
import requests
from IPython.display import Javascript

def create_gsheet():
    try:
        # ✅ Google 認證與授權
        auth.authenticate_user()
        creds, _ = default()
        creds.refresh(Request())
        headers = {"Authorization": f"Bearer {creds.token}"}
        email_response = requests.get("https://www.googleapis.com/oauth2/v1/userinfo", headers=headers)
        email = email_response.json().get("email")
        if not email:
            raise Exception("❌ 無法取得登入的 Google Email，請確保已授權 Colab 存取 Google Drive！")
        print(f"✅ 目前登入帳號：{email}")

        # ✅ 連線 Google Sheets API 並打開或建立試算表
        client = gspread.authorize(creds)
        spreadsheet_name = GSHEET_NAME  # ← 這個變數請先定義
        try:
            spreadsheet = client.open(spreadsheet_name)
        except gspread.exceptions.SpreadsheetNotFound:
            spreadsheet = client.create(spreadsheet_name)

        # ✅ 建立或取得「說明」工作表
        sheet_name = "說明"
        try:
            worksheet = spreadsheet.worksheet(sheet_name)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="1000", cols="20")

        # ✅ 刪除所有其他工作表（只保留「說明」）
        # for sheet in spreadsheet.worksheets():
        #     if sheet.title != sheet_name:
        #         spreadsheet.del_worksheet(sheet)

        # ✅ 清空「說明」內容並寫入資料
        worksheet.clear()
        df = pd.DataFrame({
            "狀態": ["檔案建立成功！"],
            "建立時間": [pd.Timestamp.now()],
            "說明": ["此工作表為預設建立，請在後續步驟中寫入資料。"]
        })
        set_with_dataframe(worksheet, df, row=1, col=1)

        print(f"✅ 已建立唯一工作表「{sheet_name}」並寫入資料")
        print(f"📋 試算表網址：{spreadsheet.url}")

    except Exception as e:
        error_message = f"❌ 發生錯誤：{str(e)}"
        display(Javascript(f'alert("{error_message}")'))
        print(error_message)
        raise

    return spreadsheet.url

# ✅ 執行建立，只保留「說明」工作表
spreadsheet_url = create_gsheet()
SPREADSHEET_ID_GSHEET = spreadsheet_url.split("/d/")[1].split("/")[0]

✅ 目前登入帳號：kevin7261@gmail.com
✅ 已建立唯一工作表「說明」並寫入資料
📋 試算表網址：https://docs.google.com/spreadsheets/d/1vYyoq0Vf07kuWJU0Rg375jpHT9r9WzA2jiCPl3V3Oi4


## [3] 檔案處理



### 3.1 CSV檔案處理

In [ ]:
# @title write_df_to_gsheet

from google.colab import auth  # 驗證 Google Colab 使用者
import gspread  # 操作 Google Sheets
from google.auth import default  # 取得授權憑證
import pandas as pd  # 處理表格資料
from gspread_dataframe import set_with_dataframe  # 寫入 DataFrame 到工作表

BATCH_SIZE = 10000  # 每批寫入的資料筆數

def write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df, sheet_name, include_index=False):
    from google.colab import auth
    import gspread
    from google.auth import default
    from gspread_dataframe import set_with_dataframe
    import pandas as pd

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # ✅ 安全填補文字欄位的 NaN，並轉為 string 避免 FutureWarning
    df = df.copy()
    str_cols = df.select_dtypes(include=["object", "string"]).columns
    df[str_cols] = df[str_cols].fillna("").astype("string")

    try:
        sh = gc.open_by_key(SPREADSHEET_ID_GSHEET)
        worksheet = sh.worksheet(sheet_name)
        print(f"✅ 工作表 '{sheet_name}' 已存在，將更新資料...")
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sh.add_worksheet(title=sheet_name, rows="1000", cols="20")
        print(f"✅ 新增工作表 '{sheet_name}'")

    current_row = 1

    for start in range(0, len(df), BATCH_SIZE):
        end = min(start + BATCH_SIZE, len(df))
        batch_df = df.iloc[start:end]

        set_with_dataframe(
            worksheet,
            batch_df,
            row=current_row,
            col=1,
            include_index=include_index,
            include_column_header=(current_row == 1)
        )

        print(f"✅ 已寫入 {start+1} 到 {end} 行資料")

        if current_row == 1:
            current_row += len(batch_df) + 1
        else:
            current_row += len(batch_df)

    worksheet.freeze(rows=1)
    worksheet.freeze(cols=1)

    print(f"✅ {sheet_name} 已成功分批寫入 Google Sheet！\n")

In [ ]:
# @title read_gsheet

import gspread
from google.colab import auth
from google.auth import default
from google.auth.transport.requests import Request
import pandas as pd
from pyproj import Transformer

def read_gsheet(SPREADSHEET_ID_GSHEET, year=None, county=None, township=None):
    try:
        # ✅ Google 授權
        auth.authenticate_user()
        creds, _ = default()
        creds.refresh(Request())
        gc = gspread.authorize(creds)

        # ✅ 開啟 Google Sheet
        sh = gc.open_by_key(SPREADSHEET_ID_GSHEET)

        # ✅ 取得工作表
        try:
            worksheet = sh.worksheet("原始資料")
        except gspread.exceptions.WorksheetNotFound:
            print("❌ 找不到工作表『原始資料』")
            return pd.DataFrame()

        # ✅ 讀取為 DataFrame
        df = pd.DataFrame(worksheet.get_all_records())

        # ✅ 處理年份
        df["發病日"] = df["發病日"].astype(str)
        df["year"] = df["發病日"].str[:4]
        if year is not None:
            year = str(year)
            df = df[df["year"] == year].copy()

        # ✅ 處理縣市
        if county is not None:
            df = df[df["居住縣市"] == county].copy()

        # ✅ 處理鄉鎮（支援陣列）
        if township is not None:
            df = df[df["居住鄉鎮"].isin(township)].copy()

        # ✅ 欄位處理
        df = df.replace('', pd.NA).infer_objects(copy=False)
        df["name"] = df["最小統計區"]
        df["longitude"] = df["最小統計區中心點X"]
        df["latitude"] = df["最小統計區中心點Y"]
        df["county"] = df["居住縣市"]
        df["township"] = df["居住鄉鎮"]
        df["village"] = df["居住村里"]

        # ✅ 經緯度轉換
        df = df.dropna(subset=["latitude", "longitude"])
        df["latitude"] = df["latitude"].astype(float)
        df["longitude"] = df["longitude"].astype(float)

        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3826", always_xy=True)
        df[["x", "y"]] = df.apply(
            lambda row: pd.Series(transformer.transform(row["longitude"], row["latitude"])),
            axis=1
        )

        # ✅ 欄位排序
        df = df[[
            "name", "latitude", "longitude", "x", "y", "year", "county", "township", "village"
        ] + [col for col in df.columns if col not in {
            "name", "latitude", "longitude", "x", "y", "year", "county", "township", "village"
        }]]

        print(f"✅ 擷取完成：{len(df)} 筆資料（{year if year else '全部年份'}）")
        return df

    except Exception as e:
        print(f"❌ 發生錯誤：{str(e)}")
        raise

In [ ]:
# @title process_distinct_point

from pyproj import Transformer  # 用於座標轉換
import pandas as pd  # 用於資料處理
import numpy as np  # 數值處理

def process_distinct_point(df, SPREADSHEET_ID_GSHEET=None, sheet_name=None):

    # ✅ 欄位定義
    x_col = "最小統計區中心點X"
    y_col = "最小統計區中心點Y"
    zone_col = "最小統計區"

    # ✅ 資料清理：將空字串轉為 NA，刪除缺值
    df_map = df.copy()
    df_map[x_col] = df_map[x_col].replace('', pd.NA)
    df_map[y_col] = df_map[y_col].replace('', pd.NA)
    df_map = df_map.dropna(subset=[x_col, y_col])
    df_map[x_col] = df_map[x_col].astype(str)
    df_map[y_col] = df_map[y_col].astype(str)

    # ✅ 統計各最小統計區的出現次數
    grouped = (
        df_map
        .groupby([zone_col, x_col, y_col])
        .size()
        .reset_index(name="count")
    )

    # ✅ 改欄位名稱統一格式
    grouped = grouped.rename(columns={
        zone_col: "name",
        y_col: "latitude",
        x_col: "longitude"
    })

    # ✅ 經緯度轉為 float 格式
    grouped["latitude"] = grouped["latitude"].astype(float)
    grouped["longitude"] = grouped["longitude"].astype(float)

    # ✅ 建立座標轉換器：WGS84 ➜ TWD97（EPSG:4326 ➜ EPSG:3826）
    transformer = Transformer.from_crs(crs_from="EPSG:4326", crs_to="EPSG:3826", always_xy=True)

    # ✅ 將經緯度轉換為 TWD97 平面座標
    grouped[["x", "y"]] = grouped.apply(
        lambda row: pd.Series(transformer.transform(row["longitude"], row["latitude"])),
        axis=1
    )

    # ✅ 根據出現次數由大到小排序並整理欄位順序
    result_df = grouped[["name", "latitude", "longitude", "x", "y", "count"]].sort_values("count", ascending=False)

    # ✅ 若指定了 SPREADSHEET_ID_GSHEET 與 sheet_name，則寫入 Google Sheet
    if SPREADSHEET_ID_GSHEET and sheet_name:
        df_to_write = result_df.copy()
        df_to_write[["latitude", "longitude", "x", "y", "count"]] = df_to_write[["latitude", "longitude", "x", "y", "count"]].astype(str)
        write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df_to_write, sheet_name)

    return result_df


In [ ]:
# @title 讀取CSV檔案寫入GSheet

df_all = pd.read_csv(DENGUE_DAILY_CSV_PATH, on_bad_lines='skip', engine="python")

write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df_all, "原始資料", include_index=False)

✅ 工作表 '原始資料' 已存在，將更新資料...
✅ 已寫入 1 到 10000 行資料
✅ 已寫入 10001 到 20000 行資料
✅ 已寫入 20001 到 30000 行資料
✅ 已寫入 30001 到 40000 行資料
✅ 已寫入 40001 到 50000 行資料
✅ 已寫入 50001 到 60000 行資料
✅ 已寫入 60001 到 70000 行資料
✅ 已寫入 70001 到 80000 行資料
✅ 已寫入 80001 到 90000 行資料
✅ 已寫入 90001 到 100000 行資料
✅ 已寫入 100001 到 107334 行資料
✅ 原始資料 已成功分批寫入 Google Sheet！



In [ ]:
# @title 讀取所需資料與寫入GSheet

df_15_tainan = read_gsheet(SPREADSHEET_ID_GSHEET, 2015, county="台南市", township=TOWNSHIPS_LIST)

write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df_15_tainan, "15_台南市區", include_index=False)

<ipython-input-6-8ff0c64d135a>:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', pd.NA).infer_objects(copy=False)


✅ 擷取完成：18656 筆資料（2015）


<ipython-input-5-4a44b7171640>:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[str_cols] = df[str_cols].fillna("").astype("string")


✅ 工作表 '15_台南市區' 已存在，將更新資料...
✅ 已寫入 1 到 10000 行資料
✅ 已寫入 10001 到 18656 行資料
✅ 15_台南市區 已成功分批寫入 Google Sheet！



In [ ]:
# @title 讀取所需資料與寫入GSheet (合併位置)

df_all_distinct = process_distinct_point(df_all)
df_15_tainan_distinct = process_distinct_point(df_15_tainan)

write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df_all_distinct, "原始資料_合併位置", include_index=False)
write_df_to_gsheet(SPREADSHEET_ID_GSHEET, df_15_tainan_distinct, "15_台南市區_合併位置", include_index=False)

✅ 工作表 '原始資料_合併位置' 已存在，將更新資料...
✅ 已寫入 1 到 10000 行資料
✅ 已寫入 10001 到 20000 行資料
✅ 已寫入 20001 到 22865 行資料
✅ 原始資料_合併位置 已成功分批寫入 Google Sheet！

✅ 工作表 '15_台南市區_合併位置' 已存在，將更新資料...
✅ 已寫入 1 到 3937 行資料
✅ 15_台南市區_合併位置 已成功分批寫入 Google Sheet！



### 3.2 SHP檔案處理

In [ ]:
# @title read_shape_file

import geopandas as gpd
import matplotlib.pyplot as plt
import gdown
import os
import zipfile

SPREADSHEET_ID_SHP = SHP_FILE_PATH.split("/d/")[1].split("/")[0]

def read_shape_file():
    # ✅ 建立工作資料夾
    os.makedirs("shapefiles", exist_ok=True)

    # ✅ 設定 ZIP 檔案名稱
    zip_path = f"{PROJECT_NAME}.zip"

    # ✅ 下載原始 ZIP
    gdown.download(url=f"https://drive.google.com/uc?id={SPREADSHEET_ID_SHP}", output=zip_path, quiet=True)

    # ✅ 解壓縮
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("shapefiles")

    # ✅ 尋找 .shp 檔
    shp_files = [f for f in os.listdir("shapefiles") if f.endswith(".shp")]
    if not shp_files:
        raise RuntimeError("❌ 無法找到 .shp 檔")
    shp_path = os.path.join("shapefiles", shp_files[0])

    # ✅ 讀取 Shapefile，指定 cp950 編碼以支援繁中
    gdf = gpd.read_file(shp_path, encoding="cp950")

    # ✅ 篩選六區
    filter_townships = TOWNSHIPS_LIST
    gdf = gdf[gdf["TOWN"].isin(filter_townships)].copy()

    # ✅ 指定儲存名稱與路徑（PROJECT_NAME.shp）
    base_name = PROJECT_NAME
    out_dir = "shapefiles"
    out_shp_path = os.path.join(out_dir, f"{base_name}.shp")

    # ✅ 儲存為新的 Shapefile
    gdf.to_file(out_shp_path, encoding="cp950")

    # ✅ 準備壓縮所有相關檔案（.shp, .shx, .dbf, .prj）
    extensions = [".shp", ".shx", ".dbf"]
    # 若有原始 prj 就複製過來
    prj_path = os.path.splitext(shp_path)[0] + ".prj"
    if os.path.exists(prj_path):
        with open(prj_path, "rb") as src, open(os.path.join(out_dir, f"{base_name}.prj"), "wb") as dst:
            dst.write(src.read())
        extensions.append(".prj")

    # ✅ 建立 ZIP（PROJECT_NAME.zip）
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for ext in extensions:
            file_path = os.path.join(out_dir, f"{base_name}{ext}")
            zipf.write(file_path, arcname=f"{base_name}{ext}")

    print(f"✅ 已儲存過濾後六區資料並壓縮為：{zip_path}")
    print("📋 前幾筆資料預覽：")
    print(gdf.head())

    return gdf

In [ ]:
# @title upload_to_drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# ✅ 取得 Google Drive 資料夾 ID
SPREADSHEET_ID_GDRIVE = GDRIVE_PATH.split("/folders/")[1].split("?")[0]

def upload_to_drive(file_path):
    # ✅ 授權與初始化
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # ✅ 檔案路徑與檔名
    source_path = os.path.join(os.getcwd(), file_path)
    filename = os.path.basename(file_path)

    # ✅ 搜尋該資料夾中是否有同名檔案 → 若有就刪除
    file_list = drive.ListFile({
        'q': f"'{SPREADSHEET_ID_GDRIVE}' in parents and trashed = false and title = '{filename}'"
    }).GetList()

    for f in file_list:
        f.Delete()
        print(f"🗑️ 已刪除舊檔案：{f['title']} (ID: {f['id']})")

    # ✅ 建立新檔並上傳
    upload_file = drive.CreateFile({
        'title': filename,
        'parents': [{'id': SPREADSHEET_ID_GDRIVE}]
    })
    upload_file.SetContentFile(source_path)
    upload_file.Upload()

    print(f"✅ 成功上傳：{filename} 到 Google Drive 資料夾（ID: {SPREADSHEET_ID_GDRIVE}）")
    print(f"📁 連結：https://drive.google.com/file/d/{upload_file['id']}/view?usp=sharing")

In [ ]:
# @title convert_shp_to_geojson

def convert_shp_to_geojson(gdf):

    gdf.to_file(f"{PROJECT_NAME}.geojson", driver="GeoJSON")

In [ ]:
# @title 讀取SHP檔案與上傳到GDrive

gdf = read_shape_file()

upload_to_drive(f"{PROJECT_NAME}.zip")

/usr/local/lib/python3.11/dist-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


✅ 已儲存過濾後六區資料並壓縮為：15_台南市區_合併位置.zip
📋 前幾筆資料預覽：
      U_ID       CODEBASE         CODE1     CODE2   TOWN_ID TOWN COUNTY_ID  \
1485  5144  A6733-0874-00  A6733-43-012  A6733-43  67000330   南區     67000   
1486  5145  A6733-0878-00  A6733-44-006  A6733-44  67000330   南區     67000   
1487  5146  A6733-0876-00  A6733-43-012  A6733-43  67000330   南區     67000   
1574  4736  A6732-0998-00  A6732-60-005  A6732-60  67000320   東區     67000   
1575  4737  A6733-0546-00  A6733-28-006  A6733-28  67000330   南區     67000   

     COUNTY             X             Y         AREA  \
1485    臺南市  166172.65021  2.535587e+06  25735.24733   
1486    臺南市  165903.12757  2.535458e+06  73573.78885   
1487    臺南市  166223.37937  2.535525e+06  20277.53652   
1574    臺南市  171126.85822  2.540743e+06  12151.50679   
1575    臺南市  166383.60272  2.540732e+06  24429.37370   

                                               geometry  
1485  POLYGON Z ((166321.261 2535692.982 0, 166319 2...  
1486  POLYGON Z ((165968.38 25356

In [ ]:
# @title 轉換SHP檔案為geojson與上傳到GDrive

convert_shp_to_geojson(gdf)

upload_to_drive(f"{PROJECT_NAME}.geojson")

/usr/local/lib/python3.11/dist-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


🗑️ 已刪除舊檔案：15_台南市區_合併位置.geojson (ID: 1ppzFLBD6pd60n2qvMSiw9gY8r45ZzGAD)
✅ 成功上傳：15_台南市區_合併位置.geojson 到 Google Drive 資料夾（ID: 1g39YQg2LLE5nf1YVDO4SUhgwa8VUaETF）
📁 連結：https://drive.google.com/file/d/1djyIaLyGPCoJNHt4Bgo-K6YZcr-5Mtys/view?usp=sharing
